## Бенчмарк для тестирования правильности ответов получаемых из RAG

### Методология

 Бенчмарк тестирует качество ответов RAG-системы путем сравнения с эталонными ответами:

1. **Загрузка вопросов** - из YAML файла с эталонными ответами
2. **Генерация ответов** - через RAG-систему с использованием Qdrant + OpenAI
3. **Оценка корректности** - автоматическое сравнение с эталоном через GPT-4
4. **Метрики** - подсчет процента правильных, частично правильных и неправильных ответов
5. **Логирование** - сохранение результатов в JSON для анализа


In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct
from openai import OpenAI
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os


In [ ]:
# Settings 
QDRANT_URL = os.getenv("QDRANT_URL")
QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
COLLECTION_NAME = os.getenv("COLLECTION_NAME")

# Client Initialization
client = OpenAI(api_key=OPENAI_API_KEY)
qdrant = QdrantClient(url=QDRANT_URL, api_key=QDRANT_API_KEY)


In [ ]:
def ask_qdrant(question):
    """
    Выполняет поиск релевантной информации в векторной базе данных Qdrant и генерирует ответ на вопрос.
    
    Функция принимает вопрос пользователя, создает его векторное представление с помощью OpenAI embeddings,
    выполняет семантический поиск в коллекции Qdrant, собирает найденные фрагменты текста в контекст
    и генерирует ответ с помощью GPT-4o-mini на основе только найденной информации.
    
    Args:
        question (str): Вопрос пользователя на русском языке
        
    Returns:
        str: Сгенерированный ответ на основе найденных в базе данных фрагментов,
             включающий ссылки на источники информации
             
    Example:
        >>> answer = ask_qdrant("Что такое лептоспироз?")
        >>> print(answer)
        'Лептоспироз — это инфекционная болезнь...'
    """
    query_embedding = client.embeddings.create(
        model="text-embedding-3-small",
        input=question
    ).data[0].embedding

    # Используем новый API query_points
    results = qdrant.query_points(
        collection_name=COLLECTION_NAME,
        query=query_embedding,
        limit=40
    )

    # Собираем контекст и источники
    context_parts = []
    sources = set()
    
    for point in results.points:
        context_parts.append(point.payload["text"])
        filename = point.payload.get("filename", "Unknown")
        page = point.payload.get("page_number", -1)
        sources.add(f"{filename} (стр. {page})" if page != -1 else filename)

    context = "\n\n".join(context_parts)

    prompt = f"""Системное сообщение:
                Ты — опытный ветеринарный консультант. У тебя есть доступ к базе знаний — текстам, документам и фрагментам, которые были извлечены по запросу пользователя.  
                Когда пользователь задаёт вопрос, ты:  
                - извлекаешь релевантные фрагменты из этой базы;  
                - используешь **только** информацию, содержащуюся в этих фрагментах;  
                - **не** добавляешь никаких внешних знаний или догадок за пределами базы;  
                - если база не содержит достаточной информации для ответа — честно говоришь об этом.

                Инструкции к ответу:  
                1. Прочти вопрос пользователя.  
                2. Проверь, какие фрагменты базы связаны с этим вопросом.  
                3. На основе этих фрагментов сформулируй ответ.  
                4. Если фрагментов недостаточно — скажи: «Извините, но у меня нет достаточной информации в базе, чтобы ответить на этот вопрос».  
                5. Ответ должен быть на русском языке, понятным, профессиональным.  
                6. Обязательно укажи источники: название документа / файл / страница, откуда взят фрагмент.

                Пользовательский вопрос:  
                {question}

                Фрагменты контекста:  
                {context}

                Источники информации:
                {chr(10).join(f"• {source}" for source in sorted(sources))}"""

    answer = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    )

    return answer.choices[0].message.content


In [28]:
import yaml

# Read questions from YAML file
yaml_path = 'benchmark.yaml'  # File is in same directory as notebook
with open(yaml_path, 'r', encoding='utf-8') as file:
    benchmark_data = yaml.safe_load(file)

questions = [q['question'] for q in benchmark_data['questions']]
answers = [q['answer'] for q in benchmark_data['questions']]


In [ ]:
import datetime
import json

def ai_judge(question, ai_answer, correct_answer):
    """
    Оценивает качество ответа ИИ на ветеринарный вопрос с помощью экспертной модели.
    
    Функция использует GPT-4o-mini в качестве судьи для сравнения ответа ИИ с эталонным ответом.
    Оценка производится по критериям фактической точности, полноты, соответствия медицинской
    терминологии и практической применимости.
    
    Args:
        question (str): Исходный ветеринарный вопрос
        ai_answer (str): Ответ, сгенерированный ИИ-системой
        correct_answer (str): Эталонный (правильный) ответ для сравнения
        
    Returns:
        str: JSON-строка с результатом оценки в одной из категорий:
             - "Correct" - ответ полностью или в основном соответствует эталону
             - "Partially correct" - ответ содержит верную информацию, но неполный 
               или имеет незначительные неточности
             - "Incorrect" - ответ содержит существенные ошибки или не соответствует эталону
             
    Example:
        >>> result = ai_judge("Что такое лептоспироз?", "Инфекционная болезнь...", "Правильный ответ")
        >>> print(result)
        '{"result":"Correct"}'
    """
    
    comparison_prompt = f"""Ты - эксперт по ветеринарии. Твоя задача - оценить качество ответа искусственного интеллекта на ветеринарный вопрос.

    Вопрос: {question}

    Ответ ИИ: {ai_answer}

    Эталонный ответ (правильный): {correct_answer}

    Проанализируй ответ ИИ и сравни его с эталонным ответом. Учитывай:
    - Фактическую точность информации
    - Полноту ответа
    - Соответствие медицинской терминологии
    - Практическую применимость

    Дай оценку по одной из трех категорий:
    1. Correct - ответ полностью или в основном соответствует эталону
    2. Partially correct - ответ содержит верную информацию, но неполный или имеет незначительные неточности
    3. Incorrect - ответ содержит существенные ошибки или не соответствует эталону
    """

    from pydantic import BaseModel, Field
    
    class EvaluationResult(BaseModel):
        result: str = Field(description="Result in format 'Correct/Partially correct/Incorrect'")
    
    comparison = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": comparison_prompt}],
        response_format=EvaluationResult
    )
    
    return comparison.choices[0].message.content

In [31]:
def log_question_and_answers(question, ai_answer, correct_answer, correctness):
    print("\n📝 Вопрос:", question)
    print(f"🔍 Ответ модели: {ai_answer}")
    print(f"🔍 Правильный ответ: {correct_answer}")

    # Логирование в файл
    log_entry = {
        "timestamp": datetime.datetime.now().isoformat(),
        "question": question,
        "ai_answer": ai_answer,
        "correct_answer": correct_answer,
        "correctness": correctness
    }
    
    with open("benchmark_log.json", "a", encoding="utf-8") as log_file:
        log_file.write(json.dumps(log_entry, ensure_ascii=False) + "\n")

def evaluate_answers():
    correct_count = 0
    partially_correct_count = 0
    incorrect_count = 0
    
    for question, answer in zip(questions, answers):
        ai_answer, correct_answer = ask_qdrant(question), answer

        correctness = ai_judge(question, ai_answer, correct_answer)
        log_question_and_answers(question, ai_answer, correct_answer, correctness)
        
        # Подсчёт результатов
        if "Correct" in correctness and "Partially" not in correctness:
            correct_count += 1
        elif "Partially correct" in correctness:
            partially_correct_count += 1
        else:
            incorrect_count += 1
    
    # Вывод итоговой статистики
    total = correct_count + partially_correct_count + incorrect_count
    print(f"\n📊 Итоговая статистика:")
    print(f"✅ Правильных ответов: {correct_count} ({correct_count/total*100:.1f}%)")
    print(f"🟡 Частично правильных: {partially_correct_count} ({partially_correct_count/total*100:.1f}%)")
    print(f"❌ Неправильных ответов: {incorrect_count} ({incorrect_count/total*100:.1f}%)")
    print(f"📈 Общая точность: {(correct_count + partially_correct_count*0.5)/total*100:.1f}%")
    
    # Логирование итоговой статистики в файл
    summary_log = {
        "timestamp": datetime.datetime.now().isoformat(),
        "summary": {
            "correct_count": correct_count,
            "partially_correct_count": partially_correct_count,
            "incorrect_count": incorrect_count,
            "total": total,
            "correct_percentage": correct_count/total*100,
            "partially_correct_percentage": partially_correct_count/total*100,
            "incorrect_percentage": incorrect_count/total*100,
            "overall_accuracy": (correct_count + partially_correct_count*0.5)/total*100
        }
    }
    
    with open("benchmark_log.json", "a", encoding="utf-8") as log_file:
        log_file.write(json.dumps(summary_log, ensure_ascii=False) + "\n")

# Run evaluation
evaluate_answers()


📝 Вопрос: Что такое лепстостироз
🔍 Ответ модели: Лептоспироз — это инфекционная болезнь, обусловленная микроорганизмами рода лептоспир, которые представляют собой тонкие спиралевидные грамотрицательные бактерии. Эта болезнь может поражать множество видов животных и человека, обычно проявляясь лихорадкой и желтухой. Лептоспиры являются возбудителями, которые выделяются во внешнюю среду через мочу больных животных или носителей.

Основные носители лептоспир в природе — это грызуны (крысы, мыши, полевки) и некоторые домашние животные, такие как собаки. Лептоспироз может проявляться в различных формах, включая острые и бессимптомные носительства. При этом у многих инфекция протекает бессимптомно, и только небольшая часть животных может проявлять клинические признаки.

Заболевание характеризуется высокой сезонностью, наиболее часто случаи возникают летом, когда животные имеют доступ к водоемам. Инкубационный период составляет от 4 до 10 дней, а диагностика включает в себя как клинические, 